In [1]:
import nest_asyncio
import asyncio

nest_asyncio.apply()
asyncio.run(asyncio.sleep(1))

In [2]:
import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  array[J] real y;              // estimated treatment effects
  array[J] real<lower=0> sigma; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {
    "J": 8,
    "y": [28, 8, -3, 7, -1, 1, 18, 12],
    "sigma": [15, 10, 16, 11, 9, 11, 10, 18],
}

posterior = stan.build(schools_code, data=schools_data)
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Building...



Building: found in cache, done.Messages from stanc:
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.1 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9e-06 seconds
  100

In [3]:
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,tau,eta.1,...,eta.7,eta.8,theta.1,theta.2,theta.3,theta.4,theta.5,theta.6,theta.7,theta.8
draws,,,,,,,,,,,,,,,,,,,,,
0,-39.318141,0.920671,0.365810,4.0,15.0,0.0,44.675336,-1.740605,4.684474,0.902277,...,0.579334,-0.617825,2.486090,-5.611028,-5.779775,-0.357865,-1.955806,1.013979,0.973270,-4.634790
1,-41.582536,0.345856,0.426451,3.0,7.0,0.0,47.309642,26.690315,19.210745,1.260651,...,-0.928290,-0.487513,50.908360,2.343706,30.977975,6.819097,8.897641,23.365529,8.857171,17.324826
2,-39.710047,0.993529,0.314745,4.0,15.0,0.0,43.152225,6.768088,2.886536,-1.183813,...,0.701732,-1.227800,3.350970,7.445080,7.021265,1.628844,5.942850,5.121997,8.793664,3.224000
3,-38.348721,0.934138,0.264789,4.0,15.0,0.0,41.507243,9.702691,6.143271,-0.075165,...,1.409699,0.242889,9.240931,5.706740,16.875773,4.119568,3.625169,13.906579,18.362856,11.194824
4,-38.315584,0.983294,0.365810,3.0,15.0,0.0,41.739941,8.288997,7.280877,-0.080521,...,0.394873,-0.513911,7.702736,1.090430,5.292709,19.218223,10.061036,12.453598,11.164019,4.547271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-36.796534,0.986834,0.264789,3.0,15.0,0.0,39.830829,5.702870,8.464227,0.848888,...,1.247353,-0.863167,12.888053,7.134846,-0.515575,15.889671,7.900419,4.918947,16.260753,-1.603172
3996,-42.266479,0.986153,0.365810,4.0,15.0,0.0,45.938739,1.254132,2.766118,2.355319,...,0.921475,-0.060084,7.769220,-0.017268,6.421191,1.714618,1.474939,-2.143509,3.803039,1.087933
3997,-39.117242,0.562689,0.426451,3.0,15.0,0.0,47.598366,6.037531,3.589295,0.941705,...,-0.076245,-0.322275,9.417589,13.636358,1.878778,5.276903,4.387419,4.037156,5.763867,4.880792
